In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

coefficient = 2.961636082259724
intercept = 0.00014390300083251257

df = pd.read_csv("/home/matt/Downloads/^GSPC (1).csv")
df.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
df.rename(columns={'Date':'date', 'Close':'close'}, inplace=True)
df['pct_change_SPX'] = df['close'].pct_change()
df['compound_SPX'] = (1 + df['pct_change_SPX']).cumprod()
df['pct_change_UPRO'] = coefficient * df['pct_change_SPX'] + intercept
df['compound_UPRO'] = (1 + df['pct_change_UPRO']).cumprod()
df.head()

,date,close,pct_change_SPX,compound_SPX,pct_change_UPRO,compound_UPRO
0,1957-03-04,44.060001,NaN,NaN,NaN,NaN
1,1957-03-05,44.220001,0.003631,1.003631,0.010899,1.010899
2,1957-03-06,44.230000,0.000226,1.003858,0.000814,1.011721
3,1957-03-07,44.209999,-0.000452,1.003404,-0.001195,1.010512
4,1957-03-08,44.070000,-0.003167,1.000227,-0.009235,1.001180


In [2]:
df.tail()

,date,close,pct_change_SPX,compound_SPX,pct_change_UPRO,compound_UPRO
15559,2018-12-21,2416.620117,-0.020588,54.848390,-0.060831,14201.702869
15560,2018-12-24,2351.100098,-0.027112,53.361326,-0.080153,13063.397650
15561,2018-12-26,2467.699951,0.049594,56.007714,0.147023,14984.011298
15562,2018-12-27,2488.830078,0.008563,56.487291,0.025503,15366.155242
15563,2018-12-28,2485.739990,-0.001242,56.417157,-0.003533,15311.863348


In [3]:
df.loc[df['compound_UPRO'] < df['compound_SPX']].tail(1)['date']

314    1958-05-29
Name: date, dtype: object

In [4]:
size = len(df.index)
UPRO_ndarray = np.zeros([size, size])
UPRO_ndarray[:, 0] = df['compound_UPRO'].values
row = 2
column = 1
for i in np.arange(size-1):
    UPRO_ndarray[row:, column] = UPRO_ndarray[row:, column-1] / UPRO_ndarray[row-1, column-1]
    row += 1
    column += 1

SPX_ndarray = np.zeros([size, size])    
SPX_ndarray[:, 0] = df['compound_SPX'].values
row = 2
column = 1
for i in np.arange(size-1):
    SPX_ndarray[row:, column] = SPX_ndarray[row:, column-1] / SPX_ndarray[row-1, column-1]
    row += 1
    column += 1

sunshine_date = np.array(['1900-01-01'] * size, dtype=object)
dates = df['date'].values
column = 0
row = size-1
for column in np.arange(size):
    while (SPX_ndarray[row, column] < UPRO_ndarray[row, column]):
        row -= 1
    sunshine_date[column] = dates[row]
    row = size-1

UPRO=pd.DataFrame(UPRO_ndarray)
UPRO['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
UPRO['sunshine_date'] = pd.to_datetime(sunshine_date, infer_datetime_format=True)
UPRO['time_to_sunshine'] = UPRO['sunshine_date'] - UPRO['date']
UPRO['sunshine_date'].describe()

count                   15564
unique                    407
top       2018-12-28 00:00:00
freq                      926
first     1957-10-22 00:00:00
last      2018-12-28 00:00:00
Name: sunshine_date, dtype: object

In [5]:
UPRO['time_to_sunshine'].describe()

count                        15564
mean     2698 days 14:46:48.943716
std      2385 days 16:31:59.882987
min                0 days 00:00:00
25%              435 days 18:00:00
50%             2101 days 00:00:00
75%             4831 days 00:00:00
max             8039 days 00:00:00
Name: time_to_sunshine, dtype: object

In [6]:
print (UPRO.loc[UPRO['time_to_sunshine'] > pd.Timedelta('8000 days')][['date', 'sunshine_date', 'time_to_sunshine']])

           date sunshine_date time_to_sunshine
7544 1987-03-06    2009-03-09        8039 days
7546 1987-03-10    2009-03-09        8035 days
7548 1987-03-12    2009-03-09        8033 days
7551 1987-03-17    2009-03-09        8028 days
7552 1987-03-18    2009-03-09        8027 days
7553 1987-03-19    2009-03-09        8026 days
7554 1987-03-20    2009-03-09        8025 days
7555 1987-03-23    2009-03-09        8022 days
7556 1987-03-24    2009-03-09        8021 days
7557 1987-03-25    2009-03-09        8020 days
7558 1987-03-26    2009-03-09        8019 days
7559 1987-03-27    2009-03-09        8018 days
7561 1987-03-31    2009-03-09        8014 days
7562 1987-04-01    2009-03-09        8013 days
7563 1987-04-02    2009-03-09        8012 days
7564 1987-04-03    2009-03-09        8011 days
7565 1987-04-06    2009-03-09        8008 days
7566 1987-04-07    2009-03-09        8007 days
7567 1987-04-08    2009-03-09        8006 days
7568 1987-04-09    2009-03-09        8005 days
7569 1987-04-